# OWL tutorial

Let's install owlready if we do not have it

In [6]:
## Uncomment if you do not have owlrl installed (you should have it installed from the RDFS tutorial)
#import sys
#!{sys.executable} -m pip install rdflib  owlready2 pandas

import pandas as pd
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL
from rdflib.namespace import DC, FOAF

from owlready2 import *

# documentation : https://pythonhosted.org/Owlready2/

Let's start with some basic information from a .CSV

In [ ]:
# if you have problems with installing pandas, just skip this block and look at file 'music.csv'
csv_file =  pd.read_csv('music.csv')
csv_file

### Creating an ontology

Initialise an empty ontology

In [9]:
onto = get_ontology("http://test.org/myonto.owl") # creates an empty ontology. Use the namespace you like!

#### Create classes and subclasses

Let's add some classes to the ontology

In [10]:
# every class is a subclass of owl:Thing!

class Person(Thing):
    namespace = onto
    
class Band(Thing):
    namespace = onto
        
class Instrument(Thing):
    namespace = onto

class Location(Thing) :
    namespace = onto

In [11]:
# add a class that is a subclass    
class Artist(Person):
    pass 
    # no need to specify namespace here, which is derived from Person

    
# let's check the superclasses
print(Artist.ancestors())

{myonto.Person, myonto.Artist, owl.Thing}


In [12]:
# print the list of the classes in the ontology

print(list(onto.classes()))

[myonto.Person, myonto.Band, myonto.Instrument, myonto.Location, myonto.Artist]


#### Create object properties
Properties of type owl:ObjectProperty have only non-literals as range, 
They are rdfs:subPropertyOf owl:topObjectProperty

In [13]:
class plays_instrument(ObjectProperty): 
    domain = [Artist]
    range = [Instrument]
    namespace = onto
    pass
        
class has_fan(ObjectProperty, Artist >> Person) : # another way of specifying domain and range
    namespace = onto

class born_in(Person >> Location ):
    namespace = onto

class has_artist(ObjectProperty, Band >> Artist): 
    namespace = onto

Let's add properties with restrictions too

In [14]:
# properties with restrictions
class located_in(ObjectProperty, TransitiveProperty): # if <A locatedin B>, and <B locatedin C>, infer <A locatedin C> 
    namespace = onto
    domain = [Location]
    range = [Location]
    
class plays_in(ObjectProperty, Artist >> Band): 
    inverse_property =  has_artist # if <A plays_in B>, infer <B hasArtist A>
    namespace = onto
    
class is_fan(Person >> Artist): 
    inverse_property = has_fan 
    namespace = onto
    
class plays_with(ObjectProperty, SymmetricProperty): #  if <A plays_with B>, infer <B plays_with A>
    domain = [Artist]
    range = [Artist]   
    namespace = onto

#### Create datatype properties
Properties of type owl:DatatypeProperty have only literals as range,
They are rdfs:subPropertyOf owl:topDatatypeProperty

In [15]:
# datatype properties 
class birth_date(DataProperty, FunctionalProperty): #every Person has only one birth date
    namespace = onto
    domain = [Person]
    range = [str] # we will use strings for the time being
        
class age(DataProperty, FunctionalProperty):
    namespace = onto
    domain = [Person]
    range = [int] 

In [16]:
#print all properties
list(onto.properties()) # can also return .object_properties() or .data_properties()

[myonto.birth_date,
 myonto.age,
 myonto.plays_instrument,
 myonto.has_fan,
 myonto.born_in,
 myonto.has_artist,
 myonto.located_in,
 myonto.plays_in,
 myonto.is_fan,
 myonto.plays_with]

#### Adding class restrictions 
Class Restictions are special owl:Classes

In [18]:
# restriction over Band : a band must have minimum 2 artists
Band.is_a.append(has_artist.min(2,Artist)) 

# restriction over Artist : someone that plays at least 1 instrument
Artist.is_a.append(plays_instrument.min(1))


# restrictions can also be specified in a class definition directly
class Fan(Person): 
    equivalent_to = [ Person & is_fan.some(Artist)]  

class SoloArtist(Artist):
    equivalent_to = [ Artist & Not(plays_in.some(Band))]

class MusicPerformer(Thing): 
    namespace=onto
    equivalent_to = [ Band | SoloArtist ]  
 

Let's add some disjointness between classes

In [ ]:
# in this world, you cannot be a Band and a SoloArtist at the same time 
AllDisjoint([Band,SoloArtist])

#### Create instances (individuals)
Instances in OWL are called individuals!

In [20]:
biffy = Band("biffy_clyro")
simon = Artist("simon_neil", namespace = onto ) # you can also add a namespace

# creating individuals with properties 
james = Artist("james_johnston",  plays_with=[simon] )
ilaria = Person("ilaria", is_fan=[simon], birth_date="19-03-1997" ) # because I am 23 yo
    
# or adding a property to an individual directly
simon.plays_in = [biffy]
    
uk = Location('united_kingdom', namespace= onto)
scotland = Location("Scotland", namespace= onto, located_in = [uk])
edi = Location("Edinburgh", namespace= onto, located_in = [scotland])

Let's also look at disjointness in our ontology

In [26]:
guitar     = Instrument('guitar')
piano    = Instrument('piano')
vocals = Instrument('vocals')

# Assert that there is only three possible instruments in this world
Instrument.is_a.append(OneOf([guitar, piano, vocals]))

#and that all instruments are different (instance level)
AllDifferent([guitar, piano, vocals])

AllDisjoint([myonto.guitar, myonto.piano, myonto.vocals])

#### Basic ontology querying

Let's search information in our ontology

In [27]:
print("Simon's IRI : %s" % simon.iri)
print("Ilaria is born on %s" % ilaria.birth_date)
print("")  
print("Who plays with someone? \n%s"% onto.search(plays_with = "*"))
print("")
print("Search for a IRI containing 'ilaria'\n%s" % onto.search(iri = "*ilaria"))
print("")
print("Which artist has a fan?\n%s" % onto.search(type=onto.Artist, has_fan="*"))

## other Basic queries
# iri, for searching entities by its full IRI
# type, for searching Individuals of a given Class
# subclass_of, for searching subclasses of a given Class
# is_a*, for searching both Individuals and subclasses of a given Class

Simon's IRI : http://test.org/myonto.owl#simon_neil
Ilaria is born on 19-03-1997

Who plays with someone? 
[myonto.james_johnston, myonto.simon_neil]

Search for a IRI containing 'ilaria'
[myonto.ilaria]

Which artist has a fan?
[myonto.simon_neil]


In [25]:
for d in onto.disjoints(): # will print both AllDifferent (individuals) and AllDisjoint (classes)
    print(d.entities)

[myonto.Band, myonto.SoloArtist]


#### Save asserted triples

In [28]:
onto.save(file = "my_ontology_asserted.owl", format = "rdfxml") # also supported ntriples

## Inference time

Let's look at how reasoning works.


Some things (eg superclasses) are already inferred by owlready2

In [33]:
# NB : we never assert that simon and james were persons!
for p in Person.instances(): # all people :
    print(p)

print("")
for i in Instrument.instances(): # all instruments :
    print(i) 

myonto.ilaria
myonto.simon_neil
myonto.james_johnston

myonto.guitar
myonto.piano
myonto.vocals


In [34]:
print("Simon has fan : %s" % simon.has_fan) # did we specify that simon has any fan?
print("Simon plays with : %s" % simon.plays_with) # did we specify who simon plays with?

Simon has fan : [myonto.ilaria]
Simon plays with : [myonto.james_johnston]


In [35]:
# this does not work! We need a reasoner !
print("Edinburgh is located in : %s" % edi.located_in) 

Edinburgh is located in : [myonto.Scotland]


### Consistency checking

Check out a full example at https://owlready2.readthedocs.io/en/latest/reasoning.html


In [37]:
# after adding all restrictions we can run a reasoner

with onto : 
    sync_reasoner(infer_property_values = True)
    # if you remove infer_property and infer_data_property, it will only infer over Classes!

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/owlready2/hermit:/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/__/qmryvtb57hvfvr_2z74h90yr0000gn/T/tmpr0zcydiw -Y


* Owlready * Adding relation myonto.Edinburgh located_in myonto.united_kingdom


* Owlready2 * HermiT took 2.394881010055542 seconds
* Owlready * Reparenting myonto.SoloArtist: {myonto.Artist} => {myonto.MusicPerformer, myonto.Artist}
* Owlready * Reparenting myonto.Band: {owl.Thing} => {myonto.MusicPerformer}
* Owlready * Reparenting myonto.ilaria: {myonto.Person} => {myonto.Fan}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


Owlready automatically gets the results of the reasoning from HermiT (a type of reasoner) and reclassifies Individuals and Classes. 

For example, Owlready inferred 2 new superclasses for SoloArtist : MusicPerformer and Artist .

In [38]:
# let's look at its superclasses
SoloArtist.ancestors()

{myonto.Artist,
 myonto.MusicPerformer,
 myonto.Person,
 myonto.SoloArtist,
 owl.Thing}

Now we have inferred that Edinburgh is in Scotland!

In [39]:
edi.located_in # now it works :)

[myonto.Scotland, myonto.united_kingdom]

#### Querying inferred triples

- *.get_parents_of(entity)* accepts any entity (Class, property or individual), and returns the superclasses (for a class), the superproperties (for a property), or the classes (for an individual). 

- *.get_instances_of(Class)* returns the individuals that are asserted as belonging to the given Class in the ontology. (NB for obtaining all instances, independently of the ontology they are asserted in, use Class.instances()).

- *.get_children_of(entity)* returns the subclasses (or subproperties) that are asserted for the given Class or property in the ontology. (NB for obtaining all children, independently of the ontology they are asserted in, use entity.subclasses()).

In [40]:
print(onto.get_parents_of(SoloArtist))
print(onto.get_children_of(MusicPerformer))
print(onto.get_instances_of(Instrument))

[myonto.Artist, myonto.MusicPerformer]
[myonto.Band, myonto.SoloArtist]
[myonto.guitar, myonto.piano, myonto.vocals]


In [41]:
# checks for inconsistencies in the ontology
list(default_world.inconsistent_classes())

[]

#### Save inferred ontology

Once inferred all facts with can save the ontology with the asserted and inferred facts. Compare it with your ontology_asserted.owl, and check for the differences.

In [41]:
onto.save(file = "my_ontology_inferred.owl", format = "rdfxml") # also supported ntriples